In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from random import shuffle


In [2]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv",
   "monthly_gold_prices.csv"
]

In [3]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 2048,
    'accuracyThreshold': 0.5,
    'controlThreshold': 0.7
}


In [4]:
class ReflexiveMemory:
  def __init__(self, reflexSize, dimensions_dense, dimensions_sparse_sp):
    self.acKey0 = None
    self.pairs = {}
    self.tableSize  = reflexSize
    self.dimensions_dense = dimensions_dense
    self.dimensions_sparse_sp = dimensions_sparse_sp

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      if sequence_data["count"] < 256:
        sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data
        
      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = value2['time']
      if table_entries > self.tableSize:
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( self.dimensions_dense )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr


In [ ]:
class ControlUnit:
  def __init__(self, controlThreshold):
    self.anomalyRM = []
    self.anomalyTM = []
    self.anomalyNU = []
    self.anomalyCU = []
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []
    self.historyCU = []
    self.countRMCU = 0
    self.controlThreshold = controlThreshold

  def anomalyScore(self, y, x):
      if np.count_nonzero(y) != 0:
          return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
      return 1

  def compute(self, denseColumns1, sp, tm, rm):

    if rm.acKey0 is not None:

      denseColumns0 = SDR( rm.dimensions_dense )
      denseColumns0.sparse = list(map(int, rm.acKey0.split('-')))

      tm.activateDendrites(True)
      predictiveCells = tm.getPredictiveCells()

      predictiveColumns = SDR( rm.dimensions_sparse_sp )
      predictiveColumns.sparse = list(set(sorted(list(np.where(predictiveCells.dense == 1)[0]))))

      reflexiveColumns = SDR( rm.dimensions_sparse_sp )
      reflexiveCount, denseReflexiveColumns = rm.predict(denseColumns0)
      if denseReflexiveColumns is not None:
        sp.compute(denseReflexiveColumns, False, reflexiveColumns)

      activeColumns0 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns0, False, activeColumns0)

      activeColumns1 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns1, False, activeColumns1)

      self.historyRM.append( reflexiveColumns.dense )
      self.historyTM.append( predictiveColumns.dense )
      self.historyGT.append( activeColumns1.dense )

      self.anomalyNU.append(tm.anomaly)
      self.anomalyRM.append( self.anomalyScore(activeColumns1.dense, reflexiveColumns.dense) )
      self.anomalyTM.append( self.anomalyScore(activeColumns1.dense, predictiveColumns.dense) )

      #  = sum(self.anomalyRM[-8:])/len(self.anomalyRM[-8:])
      if self.anomalyRM[-1] < self.anomalyTM[-1]:
        self.anomalyCU.append( self.anomalyRM[-1] )
        self.historyCU.append( self.historyRM[-1] )
        self.countRMCU = self.countRMCU + 1
      else:
        self.anomalyCU.append( self.anomalyTM[-1] )
        self.historyCU.append( self.historyTM[-1] )


In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] , config['sp']['inputDimensions'], config['sp']['columnDimensions'])
    cu = ControlUnit( config['controlThreshold'] )

    try:
        
        for count, record in enumerate(records):
            
            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            consumption = float(record[1])
            consumptionBits = scalarEncoder.encode(consumption)
            encoding = SDR( consumptionBits )

            cu.compute(encoding, sp, tm, rm)

            rm.add(encoding)

            activeColumns = SDR( sp.getColumnDimensions() )

            tmp_tm = time.time()
            sp.compute(encoding, learn_sp, activeColumns)
            tm.compute(activeColumns, learn=learn_tm)
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    def roc_auc_score_multiclass(y_true, y_pred):
        scores = []
        for y_class in set(y_true):
            y_true_class = [True if x == y_class else False for x in y_true]
            y_pred_class = [True if x == y_class else False for x in y_pred]
            scores.append(roc_auc_score(y_true_class, y_pred_class))
        return sum(scores) / len(scores)

    def match(y, x, idx1, cu, accuracyThreshold):
        n_samples = len(y)
        score1 = cu.anomalyScore(y[idx1], x[idx1])
        if score1 > (1 - accuracyThreshold):
            idx_closest = None
            score_closest = None
            for idx2 in range(n_samples):
                score2 = cu.anomalyScore(y[idx2], x[idx1])
                if score_closest is None or score_closest > score2:
                    score_closest = score2
                    idx_closest = idx2
            return idx_closest
        return idx1

    def calculateMetricsAnomaly(anomaly_scores, config, suffix):

        metric = {}

        anomaly_probability = []
        anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])
        for anomaly_value in anomaly_scores:
            anomaly_probability.append( anomaly_history.compute(anomaly_value) )

        metric['anomaly-avg-'+suffix] = sum(anomaly_scores) / len(anomaly_scores)
        metric['anomaly-samples-'+suffix] = len(anomaly_scores)
        metric['anomaly-prob-avg-'+suffix] = np.count_nonzero(anomaly_probability) / len(anomaly_probability)

        return metric

    def calculateMetrics(Y_dataset, X_dataset, anomaly_scores, total_infe_time, config, cu, suffix):

        metric = {}

        n_samples = len(Y_dataset)

        Y_labels = list(range(n_samples))
        X_labels = [ match(Y_dataset, X_dataset, idx, cu, config['accuracyThreshold']) for idx in range(n_samples)]
        precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
        metric['total-infer-time-'+suffix] = total_infe_time
        metric['infer-time-'+suffix] = total_infe_time / len(anomaly_scores)
        metric['accuracy-'+suffix] = accuracy_score(Y_labels, X_labels)
        metric['precision-'+suffix] = precision
        metric['recall-'+suffix] = recall
        metric['fscore-'+suffix] = fscore
        metric['support-'+suffix] = support
        metric['auc-'+suffix] = roc_auc_score_multiclass(Y_labels, X_labels)

        metric.update( calculateMetricsAnomaly(anomaly_scores, config, suffix) )

        return metric
    
    tm_infer_cu = 0
    avg_infer_time_rm = tm_infer_rm / len(cu.anomalyRM)
    avg_infer_time_tm = tm_infer_tm / len(cu.anomalyTM)
    tm_infer_cu = tm_infer_cu + (avg_infer_time_rm * cu.countRMCU)
    tm_infer_cu = tm_infer_cu + (avg_infer_time_tm * (len(cu.anomalyCU) - cu.countRMCU))

    metric = {}
    metric['dataset'] = dataset
    metric['cu-rm-count'] = cu.countRMCU
    metric.update( calculateMetrics(cu.historyGT, cu.historyRM, cu.anomalyRM, tm_infer_rm, config, cu, 'rm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyTM, cu.anomalyTM, tm_infer_tm, config, cu, 'tm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyCU, cu.anomalyCU, tm_infer_cu, config, cu, 'cu') )
    metric.update( calculateMetricsAnomaly(cu.anomalyNU, config, 'nupic') )

    dataset_metrics.append(metric)
    pbar.update(1)
    # break

pbar.close()

100%|██████████| 15/15 [05:38<00:00, 22.54s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

In [8]:
table_1_features = ['dataset','accuracy-cu','accuracy-rm','accuracy-tm','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic']
df[table_1_features]

,dataset,accuracy-cu,accuracy-rm,accuracy-tm,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic
0,monthly_sp500_pca.csv,0.436929,0.183425,0.421694,0.561920,0.800570,0.575259,0.575259
1,weekly_dow_jones.csv,0.034632,0.008658,0.032708,0.935766,0.984488,0.937590,0.937590
2,weekly_nasdaq.csv,0.224038,0.076923,0.216827,0.750574,0.905529,0.756490,0.756530
3,weekly_sp500.csv,0.361365,0.138876,0.352715,0.613500,0.841336,0.621009,0.621036
4,monthly_vix_close.csv,0.980247,0.766420,0.977778,0.034417,0.311039,0.038738,0.038985
5,monthly_vix_high.csv,0.960000,0.681975,0.951605,0.056183,0.347816,0.066869,0.067116
6,monthly_vix_low.csv,0.982222,0.771605,0.978025,0.029949,0.296598,0.035590,0.035837
7,monthly_vix_open.csv,0.958519,0.677284,0.948395,0.059191,0.368052,0.072147,0.072339
8,daily_natural_gas.csv,0.994998,0.956364,0.994136,0.006032,0.064016,0.009984,0.010157
9,daily_oil_prices.csv,0.994458,0.907349,0.992651,0.016546,0.212987,0.019582,0.019700


In [9]:
table_2_features = ['dataset','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic','anomaly-prob-avg-cu','anomaly-prob-avg-rm','anomaly-prob-avg-tm','anomaly-prob-avg-nupic','anomaly-samples-cu','anomaly-samples-rm','anomaly-samples-tm','anomaly-samples-nupic']
df[table_2_features]

,dataset,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic,anomaly-prob-avg-cu,anomaly-prob-avg-rm,anomaly-prob-avg-tm,anomaly-prob-avg-nupic,anomaly-samples-cu,anomaly-samples-rm,anomaly-samples-tm,anomaly-samples-nupic
0,monthly_sp500_pca.csv,0.561920,0.800570,0.575259,0.575259,0.391225,0.391225,0.391225,0.391225,1641,1641,1641,1641
1,weekly_dow_jones.csv,0.935766,0.984488,0.937590,0.937590,0.519481,0.519481,0.519481,0.519481,2079,2079,2079,2079
2,weekly_nasdaq.csv,0.750574,0.905529,0.756490,0.756530,0.519712,0.519712,0.519712,0.519712,2080,2080,2080,2080
3,weekly_sp500.csv,0.613500,0.841336,0.621009,0.621036,0.519942,0.519942,0.519942,0.519942,2081,2081,2081,2081
4,monthly_vix_close.csv,0.034417,0.311039,0.038738,0.038985,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
5,monthly_vix_high.csv,0.056183,0.347816,0.066869,0.067116,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
6,monthly_vix_low.csv,0.029949,0.296598,0.035590,0.035837,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
7,monthly_vix_open.csv,0.059191,0.368052,0.072147,0.072339,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
8,daily_natural_gas.csv,0.006032,0.064016,0.009984,0.010157,0.827699,0.827699,0.827699,0.827699,5798,5798,5798,5798
9,daily_oil_prices.csv,0.016546,0.212987,0.019582,0.019700,0.879639,0.879639,0.879639,0.879639,8300,8300,8300,8300


In [10]:
table_3_features = ['dataset','infer-time-cu','infer-time-rm','infer-time-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_3_features]

,dataset,infer-time-cu,infer-time-rm,infer-time-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,0.000331,0.000011,0.000358,0.543057,0.587501,0.017706
1,weekly_dow_jones.csv,0.000326,0.000009,0.000332,0.678259,0.690207,0.018867
2,weekly_nasdaq.csv,0.000364,0.000010,0.000378,0.757149,0.786623,0.020311
3,weekly_sp500.csv,0.000326,0.000010,0.000345,0.678511,0.717359,0.020451
4,monthly_vix_close.csv,0.000232,0.000010,0.000237,0.937579,0.961197,0.041453
5,monthly_vix_high.csv,0.000315,0.000011,0.000332,1.275871,1.343476,0.045851
6,monthly_vix_low.csv,0.000218,0.000010,0.000224,0.883162,0.907133,0.040342
7,monthly_vix_open.csv,0.000329,0.000011,0.000351,1.331382,1.422506,0.045442
8,daily_natural_gas.csv,0.000065,0.000007,0.000066,0.375524,0.382633,0.041988
9,daily_oil_prices.csv,0.000122,0.000008,0.000124,1.014997,1.033329,0.070342


In [11]:
df['cu-infer-speedup'] = 1 - (df['total-infer-time-cu'] / df['total-infer-time-tm'])
df['cu-accuracy-improvement'] = df['accuracy-cu'] - df['accuracy-tm']
df['cu-tm-count'] = df['anomaly-samples-cu'] - df['cu-rm-count']

table_4_features = ['dataset','cu-accuracy-improvement','cu-infer-speedup','cu-rm-count','cu-tm-count','anomaly-samples-cu','infer-time-rm','infer-time-tm']
df[table_4_features]

,dataset,cu-accuracy-improvement,cu-infer-speedup,cu-rm-count,cu-tm-count,anomaly-samples-cu,infer-time-rm,infer-time-tm
0,monthly_sp500_pca.csv,0.015235,0.075650,128,1513,1641,0.000011,0.000358
1,weekly_dow_jones.csv,0.001924,0.017311,37,2042,2079,0.000009,0.000332
2,weekly_nasdaq.csv,0.007212,0.037468,80,2000,2080,0.000010,0.000378
3,weekly_sp500.csv,0.008650,0.054153,116,1965,2081,0.000010,0.000345
4,monthly_vix_close.csv,0.002469,0.024572,104,3946,4050,0.000010,0.000237
5,monthly_vix_high.csv,0.008395,0.050321,211,3839,4050,0.000011,0.000332
6,monthly_vix_low.csv,0.004198,0.026424,112,3938,4050,0.000010,0.000224
7,monthly_vix_open.csv,0.010123,0.064059,268,3782,4050,0.000011,0.000351
8,daily_natural_gas.csv,0.000862,0.018579,121,5677,5798,0.000007,0.000066
9,daily_oil_prices.csv,0.001807,0.017740,158,8142,8300,0.000008,0.000124


In [12]:
table_5_features = ['dataset'] + sorted(list(set(df.columns) - (set(table_1_features) | set(table_2_features) | set(table_3_features)  | set(table_4_features))))
df[table_5_features]

,dataset,auc-cu,auc-rm,auc-tm,fscore-cu,fscore-rm,fscore-tm,precision-cu,precision-rm,precision-tm,recall-cu,recall-rm,recall-tm,support-cu,support-rm,support-tm
0,monthly_sp500_pca.csv,0.718293,0.591463,0.710671,0.421719,0.177081,0.407297,0.415572,0.175477,0.401780,0.436929,0.183425,0.421694,None,None,None
1,weekly_dow_jones.csv,0.517084,0.504090,0.516121,0.031987,0.007889,0.030625,0.030945,0.007817,0.029822,0.034632,0.008658,0.032708,None,None,None
2,weekly_nasdaq.csv,0.611833,0.538240,0.608225,0.214393,0.073697,0.206893,0.210514,0.072777,0.203038,0.224038,0.076923,0.216827,None,None,None
3,weekly_sp500.csv,0.680529,0.569231,0.676202,0.346120,0.134744,0.337832,0.339839,0.133246,0.331726,0.361365,0.138876,0.352715,None,None,None
4,monthly_vix_close.csv,0.990121,0.883181,0.988886,0.977432,0.762283,0.974661,0.976303,0.761600,0.973373,0.980247,0.766420,0.977778,None,None,None
5,monthly_vix_high.csv,0.979995,0.840948,0.975796,0.957570,0.678861,0.948627,0.956692,0.678239,0.947485,0.960000,0.681975,0.951605,None,None,None
6,monthly_vix_low.csv,0.991109,0.885774,0.989010,0.980004,0.768019,0.975237,0.979078,0.767314,0.974114,0.982222,0.771605,0.978025,None,None,None
7,monthly_vix_open.csv,0.979254,0.838602,0.974191,0.954790,0.673303,0.944202,0.953583,0.672695,0.942800,0.958519,0.677284,0.948395,None,None,None
8,daily_natural_gas.csv,0.997499,0.978178,0.997067,0.994452,0.955515,0.993509,0.994228,0.955360,0.993264,0.994998,0.956364,0.994136,None,None,None
9,daily_oil_prices.csv,0.997229,0.953669,0.996325,0.993329,0.904038,0.991345,0.992851,0.903276,0.990837,0.994458,0.907349,0.992651,None,None,None
